In [2]:
import datasets

data = datasets.load_from_disk("all_characters")

In [14]:
import numpy as np

data_dict = {}

for i, image in enumerate(data['image']):
    data_dict[i] = np.array(image).flatten().tolist()

datasets.Dataset.save_to_disk(datasets.Dataset.from_dict({'embeddings': data_dict.values()}), "./all_characters_flattened")

Saving the dataset (1/1 shards): 100%|██████████| 5938/5938 [00:00<00:00, 122796.84 examples/s]
